# Generate demand profiles for summary dwellings

This notebook reproduces the pipeline that reads the dwelling summary parameters,
extracts the EOH2303 weather window for 9–13 February 2022, and runs the existing
hybrid heat pump optimisation to export demand profiles for each dwelling.

## 1. Configure paths and load helper module

Adjust the paths below if you have moved the data or want to write the output
to a different directory.  The helper module exposes the same functions used
by the command-line utility.

In [ ]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd()
SOURCE_DIR = REPO_ROOT / 'Codes' / 'sourcecode'
DATA_DIR = REPO_ROOT / 'Codes' / 'Data'
OUTPUT_DIR = REPO_ROOT / 'Codes' / 'Output' / 'DemandProfiles'

if str(SOURCE_DIR) not in sys.path:
    sys.path.append(str(SOURCE_DIR))

print(f'Repository root: {REPO_ROOT}')
print(f'Source directory: {SOURCE_DIR}')
print(f'Data directory: {DATA_DIR}')
print(f'Output directory: {OUTPUT_DIR}')


## 2. Load building summary and weather data

This cell reads the filtered 1R1C1P1S summary file and extracts the required
weather window from the EOH2303 dataset.

In [ ]:
import pandas as pd

import generate_demand_profiles as gdp

summary_path = DATA_DIR / '1R1C1P1S_filtered.csv'
weather_path = DATA_DIR / 'EOH2303_merged_30min.csv'
start = pd.Timestamp('2022-02-09 00:00')
end = pd.Timestamp('2022-02-13 23:30')

params = gdp.load_summary(summary_path)
weather = gdp.load_weather(weather_path, start, end)

len(params), weather.head()


## 3. Prepare optimisation inputs

The helper functions derive the sampling period, tariff profile, comfort setpoints,
and initial indoor temperature.  Adjust the parameters in the next cell if you
wish to explore different comfort tolerances or plant limits.

In [ ]:
dt_seconds = weather.index.to_series().diff().dropna().median().total_seconds()
tariff_days = int((end.normalize() - start.normalize()).days) + 1
tariff = gdp.build_tariff(start.normalize(), n_days=tariff_days, step='30min', type='cosy')
tariff = gdp.ensure_aligned_tariff(tariff, weather.index)
setpoint = gdp.derive_setpoint(weather.index)
initial_temperature = gdp.derive_initial_temperature(weather)

comfort_tolerance = 1.0  # °C
heat_pump_cop = 3.5
boiler_efficiency = 0.9
hp_capacity_w = 4_000.0
boiler_capacity_w = 24_000.0

dt_seconds, initial_temperature


## 4. Run the optimisation for each dwelling

This step may take a few minutes depending on the number of dwellings.  The
results are exported to CSV files in the output directory, and the combined
data frame is returned for further exploration in the notebook.

In [ ]:
from collections import OrderedDict

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

optimisation_results = OrderedDict()

Tout = gdp.extract_outdoor_temperature(weather)
solar_gain = gdp.extract_solar_gain(weather, dt_seconds)

for param in params:
    result = gdp.optimize_hhp_operation(
        param.R1,
        param.C1,
        param.g,
        tariff,
        Tout,
        solar_gain,
        dt_seconds,
        initial_temperature,
        T_setpoint=setpoint,
        tol=comfort_tolerance,
        COP=heat_pump_cop,
        etaB=boiler_efficiency,
        Qhp_max=hp_capacity_w,
        Qbo_max=boiler_capacity_w,
        day_ahead=False,
    )

    result = result.copy()
    result['total_heat_W'] = result['Q_hp'] + result['Q_bo']
    result['total_heat_kW'] = result['total_heat_W'] / 1000.0

    out_path = OUTPUT_DIR / f"{param.dwelling_id}_Feb09-13_2022.csv"
    result.to_csv(out_path, index_label='time')
    optimisation_results[param.dwelling_id] = result
    print(f'Saved demand profile for {param.dwelling_id} -> {out_path}')

len(optimisation_results)


## 5. Inspect one of the generated profiles

Select a dwelling ID from the dictionary keys above and display the last few rows
to verify the outputs.

In [ ]:
sample_dwelling = next(iter(optimisation_results))
optimisation_results[sample_dwelling].tail()
